<a href="https://colab.research.google.com/github/LA-LA-L/GEN/blob/main/copy_of_epub_pdf_to_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required libraries
!pip install PyPDF2 ebooklib pandas beautifulsoup4

# Upload files
from google.colab import files
uploaded = files.upload()

# Extract text based on file type
def extract_text(file_path):
    if file_path.endswith('.pdf'):
        import PyPDF2
        with open(file_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            text = ''
            for page in reader.pages:
                text += page.extract_text()
        return text

    elif file_path.endswith('.epub'):
        from ebooklib import epub
        from bs4 import BeautifulSoup  # For cleaning HTML content

        book = epub.read_epub(file_path)
        text = ''
        for item in book.get_items():
            # Check if the item is a document (chapter or content)
            if item.get_type() == 9:  # 9 corresponds to ITEM_DOCUMENT
                soup = BeautifulSoup(item.get_content(), 'html.parser')
                text += soup.get_text(separator='\n')  # Extract clean text
        return text

    else:
        raise ValueError("Unsupported file format. Please upload a PDF or EPUB file.")

# Process all uploaded files
import re
import pandas as pd

all_paragraphs = []

for file_name in uploaded.keys():
    print(f"Processing file: {file_name}")
    extracted_text = extract_text(file_name)

    # Split text into paragraphs
    paragraphs = re.split(r'\n\s*\n', extracted_text)  # Split by double newlines
    paragraphs = [p.strip() for p in paragraphs if p.strip()]  # Remove empty paragraphs

    # Add paragraphs to the list with file name as context
    for paragraph in paragraphs:
        all_paragraphs.append({'File': file_name, 'Paragraph': paragraph})

# Create a DataFrame
df = pd.DataFrame(all_paragraphs)

# Save as CSV
csv_file = 'book_data.csv'
df.to_csv(csv_file, index=False)

# Download the CSV file
from google.colab import files
files.download(csv_file)

print(f"Text extracted from all files and saved as {csv_file}")

KeyboardInterrupt: 